In [1]:
pip install powerbiclient

  Using cached powerbiclient-1.0.0-py2.py3-none-any.whl (585 kB)
  Using cached msal-1.12.0-py2.py3-none-any.whl (66 kB)
  Using cached requests-2.25.1-py2.py3-none-any.whl (61 kB)
  Using cached jupyter_ui_poll-0.1.2-py2.py3-none-any.whl (6.6 kB)
  Using cached cryptography-3.4.7-cp36-abi3-win_amd64.whl (1.6 MB)



  Attempting uninstall: requests
    Found existing installation: requests 2.24.0
    Can't uninstall 'requests'. No files were found to uninstall.


In [4]:
#https://github.com/microsoft/powerbi-jupyter/blob/main/demo/demo.ipynb
from powerbiclient import Report, models
from io import StringIO
from ipywidgets import interact
import requests
import pandas as pd

In [1]:
#Open Spark:?
import pyspark
from pyspark.sql import SparkSession
spark=SparkSession.builder.getOrCreate()
df=spark.sql("select 'spark'as hello")
df.show()

+-----+
|hello|
+-----+
|spark|
+-----+



In [2]:
#Testing Spark Query Plan
spark.sql("EXPLAIN SELECT 'Spark' as hello").first()
df.explain()

== Physical Plan ==
*(1) Project [spark AS hello#0]
+- *(1) Scan OneRowRelation[]




In [3]:
import pyspark
from pyspark.sql import SparkSession
spark=SparkSession.builder.getOrCreate()
df=spark.read.csv("trainsched.txt",header=True)
df.createOrReplaceTempView("schedule")
spark.sql("SELECT * FROM schedule WHERE station='San Jose'").show()


+--------+--------+-----+
|train_id| station| time|
+--------+--------+-----+
|     324|San Jose|9:05a|
|     217|San Jose|6:59a|
+--------+--------+-----+



In [10]:
result=spark.sql("DESCRIBE schedule")
print(result.columns)

['col_name', 'data_type', 'comment']


In [13]:
query="SELECT train_id,  min(time) as start, max(time) as end FROM schedule GROUP BY train_id"
spark.sql(query).show()

+--------+-----+-----+
|train_id|start|  end|
+--------+-----+-----+
|     217|6:06a|6:59a|
|     324|7:59a|9:05a|
+--------+-----+-----+



In [20]:
#Some basic loading tricks
from pyspark.sql.functions import lower, col
dfS=spark.read.csv('sherlock.txt')
print(dfS.first())
dfS.count()
dfS.show(10, truncate=False)
dfS.columns
df1=dfS.select(lower(col('_c0')).alias('v'))
print(df1.first())
df1.columns

Row(_c0='The Project Gutenberg EBook of The Adventures of Sherlock Holmes')
+--------------------------------------------------------------------+
|_c0                                                                 |
+--------------------------------------------------------------------+
|The Project Gutenberg EBook of The Adventures of Sherlock Holmes    |
|by Sir Arthur Conan Doyle                                           |
|(#15 in our series by Sir Arthur Conan Doyle)                       |
|Copyright laws are changing all over the world. Be sure to check the|
|copyright laws for your country before downloading or redistributing|
|this or any other Project Gutenberg eBook.                          |
|This header should be the first thing seen when viewing this Project|
|Gutenberg file.  Please do not remove it.  Do not change or edit the|
|header without written permission.                                  |
|Please read the "legal small print                                  |
+

['v']

In [42]:
#contiune testing partition as well as cleaning data
#Replacing
from pyspark.sql.functions import regexp_replace, split
df2=df1.select(regexp_replace('v','Mr\.','Mr').alias('v'))
#df2=df2.select(regexp_replace('v','don\'t','do not'))
df2.show(truncate=False)

#Tokenzing
#df3=df2.select(split('v','[]').alians('words'))
#df3.show(truncate=False)

#Split characters 
puncturation="_|.\?\!\",\'\[\]\*()"
df4=df2.select(split('v', '[%s\]'%puncturation).alias('words'))
df4.show(truncate=False)

#Exploding array
df5=df4.select(explode('words').alias('word'))
df5.show()

+--------------------------------------------------------------------+
|v                                                                   |
+--------------------------------------------------------------------+
|the project gutenberg ebook of the adventures of sherlock holmes    |
|by sir arthur conan doyle                                           |
|(#15 in our series by sir arthur conan doyle)                       |
|copyright laws are changing all over the world. be sure to check the|
|copyright laws for your country before downloading or redistributing|
|this or any other project gutenberg ebook.                          |
|this header should be the first thing seen when viewing this project|
|gutenberg file.  please do not remove it.  do not change or edit the|
|header without written permission.                                  |
|please read the "legal small print                                  |
|ebook and project gutenberg at the bottom of this file.  included is|
|impor

Py4JJavaError: An error occurred while calling o325.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 75.0 failed 1 times, most recent failure: Lost task 0.0 in stage 75.0 (TID 87) (DESKTOP-JUIOPEK executor driver): java.util.regex.PatternSyntaxException: Unclosed character class near index 20
[_|.\?\!",'\[\]\*()\]
                    ^
	at java.util.regex.Pattern.error(Unknown Source)
	at java.util.regex.Pattern.clazz(Unknown Source)
	at java.util.regex.Pattern.sequence(Unknown Source)
	at java.util.regex.Pattern.expr(Unknown Source)
	at java.util.regex.Pattern.compile(Unknown Source)
	at java.util.regex.Pattern.<init>(Unknown Source)
	at java.util.regex.Pattern.compile(Unknown Source)
	at java.lang.String.split(Unknown Source)
	at org.apache.spark.unsafe.types.UTF8String.split(UTF8String.java:1034)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at o

In [ ]:
#Partition

In [ ]:
#Caching, Loagging , and the Spark UI